# <center>Network Science</center>

## <center>Home Assignment #3: Centralities and Assortativity</center>

### <center>Student: Elena Andreeva</center>

#### <hr /> General Information

**Due Date:** 13.03.2016 23:59 <br \>
**Late submission policy:** -0.2 points per day <br \>


Please send your reports to <network.hse.2016@gmail.com> with message subject of the following structure:<br \> **[HSE Networks 2015] *{LastName}* *{First Name}* HA*{Number}***

Support your computations with figures and comments. <br \>
If you are using IPython Notebook you may use this file as a starting point of your report.<br \>
<br \>
<hr \>

## Problems

### Task 1

Compute degree centrality, Pagerank and  HubAuthorities scores for the [flickr](https://www.hse.ru/data/2016/02/24/1139618969/flickr.mat) network. 

Data contains sparse matrix A and list of user names.
This is a “denser” part of the Flickr photo sharing site friendship graph from 2006. Edge direction corresponds to friendship requests (following). Some of the links are reciprocal,others not.  

Provide top 50 names in each ranking, compare results

You can load .mat files with the following commands:

In [1]:
import scipy.io

data = scipy.io.loadmat('flickr.mat')

In [2]:
# Constructing the graph
import networkx as nx
indexes = data['A'].nonzero()
shape = data['A'].shape
G = nx.DiGraph()
G.add_nodes_from(range(0, shape[0]))

for i in range(0, len(indexes[0])):
    G.add_edge(indexes[0][i], indexes[1][i])


In [3]:
# Degree centrality top (in)
degree_centr = []
for node in G.nodes():
    degree_centr.append((data['names'][node],G.in_degree(node)))
degree_centr_in = sorted(degree_centr, key = lambda x: -x[1])[0:50]
print("Top 50 users by degree_centrality (in)")
for user in degree_centr_in:
    print("{}\t{}".format(user[1], user[0].encode("utf-8")))

Top 50 users by degree_centrality (in)
1134	awfulsara                                           
795	drp                                                 
786	*Ivan*                                              
760	antimethod                                          
740	DrJoanne                                            
725	BombDog                                             
647	Simon Pais                                          
639	deborah lattimore                                   
611	MaD GiÂ®Lâ¢â¢                                     
526	notraces                                            
526	:Nikola                                             
524	cymagen                                             
520	aleyna                                              
497	lorrainemd                                          
495	artofgold                                           
485	*starlet*                                           
484	romanlily                                   

In [4]:
# Degree centrality top (out)
degree_centr = []
for node in G.nodes():
    degree_centr.append((data['names'][node],G.out_degree(node)))
degree_centr_out = sorted(degree_centr, key = lambda x: -x[1])[0:50]
print("Top 50 users by degree_centrality (out)")
for user in degree_centr_out:
    print("{}\t{}".format(user[1], user[0].encode("utf-8")))


Top 50 users by degree_centrality (out)
185	anildash                                            
174	tozzer                                              
170	AtiRanA                                             
166	pixietart                                           
163	jakedobkin                                          
162	Buntekuh                                            
161	brainware3000                                       
161	Jakes_World                                         
160	maximolly                                           
158	Andreia Lopes                                       
157	elvis1967                                           
157	inthegan                                            
155	990000                                              
154	: Esther                                            
153	dmoore                                              
153	kathryn                                             
153	Charles Machado                             

 Ok, let's find out, how many users are in both tops.

In [5]:
dc_out = set()
for user in degree_centr_out:
    dc_out.add(user[0])
dc_in = set()
both_dc = set()
for user in degree_centr_in:
    dc_in.add(user[0])
for user in dc_out:
    if user not in dc_in:
        continue
    print(user)
    both_dc.add(user)


drp                                                 
pixietart                                           
anildash                                            
cymagen                                             


The next top is about PageRank. I guess, the PageRank top must have many users, that got in "in-degree" top, as lot's of links point on them.

In [6]:
# PageRank
alp = 0.8
pr = nx.pagerank(G, alpha=alp, max_iter=1000)
pr_top = sorted(pr.items(), key = lambda x: -x[1])[0:50]
pr_top_set = set()
print("Top 50 PageRank users")
for user in pr_top:
    print("{} {}".format(user[1], data['names'][user[0]].encode("utf-8")))
    pr_top_set.add(data['names'][user[0]])

Top 50 PageRank users
0.00186995591718 awfulsara                                           
0.0013201765653 drp                                                 
0.00127208160583 antimethod                                          
0.0012308598497 BombDog                                             
0.00113290988364 *Ivan*                                              
0.00110660270486 MaD GiÂ®Lâ¢â¢                                     
0.00101887312259 DrJoanne                                            
0.000989214537391 Simon Pais                                          
0.000947320054751 deborah lattimore                                   
0.000911100661548 cymagen                                             
0.000880914077451 :Nikola                                             
0.000851872844206 notraces                                            
0.000813980943262 lorrainemd                                          
0.000812260260259 jkottke                                       

In [7]:
# Let's check the intersection between PageRank top and degree_centralities tops
print("Intersection PageRank and degree-out ")
count = 0 
for user in pr_top_set:
    if user in dc_out:
        print user
        count += 1
print ("Users in intersection: {}".format(count))
print("\nIntersection PageRank and degree-in")
count = 0
for user in pr_top_set:
    if user in dc_in:
        print user
        count += 1
print ("Users in intersection: {}".format(count))

Intersection PageRank and degree-out 
pixietart                                           
maximolly                                           
drp                                                 
anildash                                            
cymagen                                             
Users in intersection: 5

Intersection PageRank and degree-in
hot_luscious                                        
CherryVega                                          
slowernet                                           
Mylens                                              
carf                                                
pixietart                                           
Agridulce                                           
naftalina007                                        
gu@va                                               
fraying                                             
Loobylu                                             
aleyna                                              

So, my guess was quite right - 44 users from the top of "in-degree" got into top on Page Rank. And, interesting fact - 4 users, that got into both "degree" tops got in PageRank top too!
The next step is to build top-50 Hubs and Authorities. I suppose, that our 4 users will get into both of tops. I also suppose, that Authorities and In-degrees will have a big intersection, as well as Out-degrees and Hubs. Let's check it.

In [8]:
(hubs, auths) = nx.hits(G)
hubs_top = sorted(hubs.items(), key = lambda x: -x[1])[0:50]
auths_top = sorted(auths.items(), key = lambda x: -x[1])[0:50]
hubs_set = set()

out_d_inters = 0
in_d_inters = 0
pr_inters = 0
print("Hubs top:\n")
for hub in hubs_top:
    username = data['names'][hub[0]].encode("utf-8")
    status = []
    if username in dc_out:
        status.append("Out-degree")
        out_d_inters += 1
    if username in dc_in:
        status.append("In-degree")
        in_d_inters += 1
    if username in pr_top_set:
        status.append("PageRank")
        pr_inters += 1
    status = "(" + ",".join(status) + ")" if len(status) > 0 else ""
    print("{}\t{}\t{}".format(hub[1], username.strip(), status))
    hubs_set.add(username)
print("In-degree top intersection: {}".format(in_d_inters))   
print("Out-degree top intersection: {}".format(out_d_inters))  
print("PageRank top intersection: {}".format(pr_inters))  

Hubs top:

0.000589060455139	mrpiink	
0.000585837904157	automat	
0.000577658016738	schizoo23	
0.00055872016857	lorrainemd	(In-degree,PageRank)
0.000550810713606	sgoralnick	
0.000543889060531	*starlet*	(In-degree,PageRank)
0.000536905156737	brynfoto	
0.000529979918205	liquid pixel	
0.000517658021219	noahstone	
0.000516133458287	isherwood	
0.000513472966666	Arno-MM	
0.000508661271453	scotsman0170	
0.000507479510422	WaveSpire	
0.000501112336149	tozzer	(Out-degree)
0.00049786514595	...cathzilla	
0.000495505457874	cymagen	(Out-degree,In-degree,PageRank)
0.000494654807593	ola_babe1	
0.000488947747202	DrJoanne	(In-degree,PageRank)
0.000486682876206	pablokorona	
0.000486212851927	demouser	
0.000484262629846	jallive	
0.00048411279957	macebio	
0.000482783744172	marika b	
0.000480730748293	drp	(Out-degree,In-degree,PageRank)
0.000479738112111	Osvaldo	
0.000478720696014	Crea.tive	
0.000475369730467	sheldonschwartz	
0.000472349340854	virgu	(Out-degree)
0.000471911781431	mehmetkale	(Out-degree)
0.00

In [9]:
out_d_inters = 0
in_d_inters = 0
pr_inters = 0
hubs_inters = 0
print("Auths top:\n")
for auth in auths_top:
    username = data['names'][auth[0]].encode("utf-8")
    status = []
    if username in dc_out:
        status.append("Out-degree")
        out_d_inters += 1
    if username in dc_in:
        status.append("In-degree")
        in_d_inters += 1
    if username in pr_top_set:
        status.append("PageRank")
        pr_inters += 1
    if username in hubs_set:
        status.append("Hubs")
        hubs_inters += 1
    status = "(" + ",".join(status) + ")" if len(status) > 0 else ""
    print("{}\t{}\t{}".format(auth[1], username.strip(), status))
print("In-degree top intersection: {}".format(in_d_inters))   
print("Out-degree top intersection: {}".format(out_d_inters))  
print("PageRank top intersection: {}".format(pr_inters))  
print("Hubs top intersection: {}".format(hubs_inters))  

Auths top:

0.00339652885621	awfulsara	(In-degree,PageRank)
0.00255993479745	drp	(Out-degree,In-degree,PageRank,Hubs)
0.00250711114245	DrJoanne	(In-degree,PageRank,Hubs)
0.00224604143983	*Ivan*	(In-degree,PageRank)
0.00220053873489	antimethod	(In-degree,PageRank)
0.00215780914846	BombDog	(In-degree,PageRank)
0.00204474149824	Simon Pais	(In-degree,PageRank)
0.00182896126509	deborah lattimore	(In-degree,PageRank)
0.00176300080906	cymagen	(Out-degree,In-degree,PageRank,Hubs)
0.00170858289198	:Nikola	(In-degree,PageRank)
0.00164976848217	lorrainemd	(In-degree,PageRank,Hubs)
0.00160194813583	*starlet*	(In-degree,PageRank,Hubs)
0.00160010255952	aleyna	(In-degree,PageRank)
0.00146577993629	artofgold	(In-degree,PageRank)
0.00137519240679	notraces	(In-degree,PageRank,Hubs)
0.00131934121392	romanlily	(In-degree,PageRank)
0.00130700897056	Pandarine	(In-degree,PageRank)
0.00129001994661	Mylens	(In-degree,PageRank)
0.00121194554301	Mareen Fischinger	(In-degree,PageRank,Hubs)
0.00120230686388	bernar

As we see, indeed, top-authorities intersects with top of in-degrees. That is kinda obvious. While hubs top is more interesting -- top-3 hubs were not in any other tops at all. 

### <hr /> Task 2

Here are the [Facebook friendship graphs](https://www.hse.ru/data/2016/03/05/1125656999/universities.zip) from several US universities from 2005 (one year after fb launch).

Data contains a A matrix (sparse) and a "local_info" variable, one row per node: 
a student/faculty status flag, gender, major, second major/minor (if applicable), dorm/house, year, and high school. 
Missing data is coded 0.

Compute node degree assortativity (mixining by node degree) and assortativity coefficient (modularity) for gender, major, dormitory, year, high school for all universities and compare the results.

In [24]:
import os
folder = "fb_university/"
universities = os.listdir(folder)
universities

['Berkeley13.mat',
 'Caltech36.mat',
 'Harvard1.mat',
 'Oklahoma97.mat',
 'Princeton12.mat',
 'Texas80.mat']

In [25]:
class ListTable(list):    
    def _repr_html_(self):
        html = ["<table>"]
        for row in self:
            html.append("<tr>")            
            for col in row:
                html.append("<td>{0}</td>".format(col))           
            html.append("</tr>")
        html.append("</table>")
        return ''.join(html)

In [31]:
attr_names = ["status", "gender", "major", "minor", "dorm", "year", "school"]
table = ListTable()
table.append(["university", "degree assort.", "gender", "major", "dorm", "year", "school"])

    
for uni in universities:
    print(uni)
    # Create graph from .mat
    data = scipy.io.loadmat(folder + uni)
    shape = data['local_info'].shape
    indexes = data['A'].nonzero()
    shape = data['A'].shape
    G = nx.DiGraph()
    G.add_nodes_from(range(0, shape[0]))

    for i in range(0, len(indexes[0])):
        G.add_edge(indexes[0][i], indexes[1][i])
        
    # Setting nodes attributes 
    shape = data['local_info'].shape
    attr_dicts = []
    for i in range(0, shape[1]):
        attr_dicts.append(dict())
        
    for student in range(0, shape[0]):
        for attr in range(0, shape[1]):  
            if data['local_info'][student, attr] != 0:
                attr_dicts[attr][student] = data['local_info'][student, attr]

    for i in range(0, shape[1]):
        nx.set_node_attributes(G, attr_names[i], attr_dicts[i]) 
    
    # Actually counting the coeffs
    row = []
    row.append(uni)
    row.append(nx.degree_assortativity_coefficient(G))
    row.append(nx.attribute_assortativity_coefficient(G, "gender"))
    row.append(nx.attribute_assortativity_coefficient(G, "major"))
    row.append(nx.attribute_assortativity_coefficient(G, "dorm"))
    row.append(nx.attribute_assortativity_coefficient(G, "year"))
    row.append(nx.attribute_assortativity_coefficient(G, "school"))
    table.append(row)

Berkeley13.mat
Caltech36.mat
Harvard1.mat
Oklahoma97.mat
Princeton12.mat
Texas80.mat


In [32]:
table

university,degree assort.,gender,major,dorm,year,school
Berkeley13.mat,0.0103134590889,0.0409228376393,0.058255029946,0.14727644076,0.308121015806,0.0773753653543
Caltech36.mat,-0.0652729511948,0.0463314364559,0.0347194237236,0.349153100873,0.237531904813,0.00266977930658
Harvard1.mat,0.145054887671,0.0452699773794,0.0496769283286,0.12567663505,0.421159642116,0.0239867253741
Oklahoma97.mat,0.0736732579392,0.0928372959827,0.0363790887617,0.280335243729,0.20116469147,0.139468406924
Princeton12.mat,0.091092123568,0.0506107399953,0.0543901188714,0.0946784195542,0.446744620584,0.018783655245
Texas80.mat,0.163892412814,0.0796930681412,0.0597127278334,0.204837604339,0.246194684162,0.147992848231


From the table above we see, that almost on all universities the biggest influence had the "year" attribute. That is kinda obvious - students from the same year of study are more likely to be friends, than students from different years. Only in Oklahoma and Caltech this attribute is not the first one (though it is the second) -- the same dormitory plays the most important role in these universities.
So, dormitory is also important attribute - the sudents from the same dormitory are more likely to be friends. As to the other attributes, the school plays role only in Oklahoma and Texas (I wonder, why), major and gender do not hame that much influence on connections. 
It is interesting, that node degree attribute's ole differs greatly from one university to another. The highest impact this attribute has in Texas, then - in Harvard (and node degree is more important, that event dormitory there).  In Princeton and Oklahoma node degree is twice as low, in Berkeley have almost no influence and in Caltech it has even negative impact. How very interesting...